In [12]:
import numpy as np

from PIL import Image, ImageDraw
import torch
import torchvision
from IPython.display import display
import os
np.random.seed(42)

Experimenting to make the importing data independent of file structure

In [23]:
os.path.abspath(os.path.join(os.getcwd(), os.pardir))

'/Users/telekenesis/Husky Gdrive/Coursework/7. SML_EE/Project'

In [19]:
train_Images_folder = os.path.abspath(os.path.join(os.getcwd(), os.pardir))+'/Data/Wider dataset/WIDER_train/images/'
test_Images_folder = os.path.abspath(os.path.join(os.getcwd(), os.pardir))+'/Data/Wider dataset/WIDER_val/images/'
train_bbx = os.path.abspath(os.path.join(os.getcwd(), os.pardir))+'/Data/Wider dataset/wider_face_split/wider_face_train_bbx_gt.txt'
test_bbx = os.path.abspath(os.path.join(os.getcwd(), os.pardir))+'/Data/Wider dataset/wider_face_split/wider_face_val_bbx_gt.txt'
data_folder = '~/Data/Wider dataset/Faces'

Need to remove occlusion
need to remove invalid images

In [20]:
def create_data_dict(bbx_path,images_path):
    data = {}
    with open(bbx_path, 'r') as file:
        while True:
            line = file.readline()[:-1]#reads the image path
            if not line: break
            faces = []
            n = int(file.readline())#reads the number of bounding boxes present in the image at Path(line)
            check_valid = 1
            im = Image.open(images_path+line)
            l,w,_ = np.array(im).shape
            if n>0:
                while n>0:
                    #reads bounding boxes values and types
                    v = np.fromstring(file.readline(), dtype = float, sep=' ')
                    if check_valid ==1:
                    #if not clear or not valid or contains heavy occlusion do not consider
                        if v[4]!=0 or v[7]==1 or v[8]==2:
                            check_valid = 0
                        area = v[2]*v[3]
                        if area<100:
                            check_valid = 0
                        faces.append([v[0]/w, v[1]/l, v[2]/w, v[3]/l])
                    n-=1
                if check_valid:
                    data[line]=np.array(faces)
                    print(data[line])
            else: line = file.readline()
    
    return data

In [27]:
# train_data = create_data_dict(train_bbx,train_Images_folder)

test_data = create_data_dict(test_bbx,test_Images_folder)

[[0.10839844 0.30619597 0.11914062 0.09149856]
 [0.20410156 0.25       0.06835938 0.07420749]
 [0.359375   0.1815562  0.08691406 0.09582133]
 [0.54199219 0.20317003 0.08691406 0.07204611]
 [0.69042969 0.1815562  0.08984375 0.09582133]]
[[0.48925781 0.11721612 0.27832031 0.32454212]]
[[0.046875   0.25830258 0.109375   0.19434194]
 [0.2265625  0.0295203  0.06640625 0.12300123]
 [0.6953125  0.29520295 0.06445312 0.12300123]
 [0.85742188 0.23616236 0.078125   0.13284133]]
[[0.00683594 0.34131737 0.19628906 0.29255774]
 [0.27636719 0.58169376 0.16015625 0.17792985]
 [0.6953125  0.30025663 0.28515625 0.31993157]]
[[0.25683594 0.27789934 0.11035156 0.12326769]
 [0.62011719 0.19766594 0.13085938 0.12326769]]
[[0.62792969 0.21875    0.31640625 0.36002604]]
[[0.11132812 0.19756098 0.06835938 0.12926829]
 [0.24804688 0.14390244 0.08203125 0.13658537]
 [0.4609375  0.05121951 0.08203125 0.13658537]
 [0.65039062 0.16585366 0.06835938 0.11707317]
 [0.74609375 0.13658537 0.08984375 0.13414634]]
[[0.16

[[0.06640625 0.16740088 0.05664062 0.17621145]
 [0.2421875  0.2701909  0.0625     0.15565345]
 [0.36914062 0.16740088 0.06054688 0.14096916]
 [0.52929688 0.27900147 0.07226562 0.14977974]
 [0.8125     0.19089574 0.06835938 0.17621145]]
[[0.14453125 0.22222222 0.20507812 0.4005848 ]]
[[0.18457031 0.20937042 0.04589844 0.09809663]
 [0.25097656 0.18887262 0.04394531 0.09224012]
 [0.33300781 0.22547584 0.04785156 0.09224012]
 [0.38964844 0.17715959 0.04101562 0.08052709]
 [0.45605469 0.25183016 0.04492188 0.09370425]
 [0.54296875 0.20644217 0.04101562 0.07759883]
 [0.63476562 0.19472914 0.05175781 0.10102489]
 [0.7265625  0.15373353 0.04394531 0.07027818]
 [0.78320312 0.17569546 0.05078125 0.10102489]]
[[0.64550781 0.32421875 0.046875   0.0703125 ]
 [0.57519531 0.48567708 0.04589844 0.06640625]
 [0.53222656 0.34765625 0.04394531 0.05729167]
 [0.41699219 0.3203125  0.04589844 0.0703125 ]
 [0.34179688 0.36067708 0.04589844 0.0625    ]
 [0.24414062 0.35416667 0.04394531 0.07161458]
 [0.137695

[[0.08984375 0.32565284 0.1953125  0.33794163]
 [0.515625   0.31336406 0.23828125 0.38709677]
 [0.33398438 0.25806452 0.1328125  0.25499232]
 [0.00390625 0.29185868 0.0859375  0.23348694]]
[[0.20898438 0.05511811 0.07421875 0.13385827]]
[[0.38085938 0.09895833 0.11328125 0.2109375 ]]
[[0.56152344 0.20300261 0.08203125 0.08616188]
 [0.36230469 0.23237598 0.07617188 0.07832898]]
[[0.38085938 0.08203125 0.21386719 0.18359375]]
[[0.40039062 0.14492754 0.22851562 0.27706735]]
[[0.33105469 0.30641509 0.20996094 0.26339623]]
[[0.109375   0.23809524 0.51855469 0.57435897]]
[[0.47460938 0.15619694 0.1640625  0.40067912]]
[[0.08007812 0.29861111 0.07226562 0.16666667]
 [0.1796875  0.27430556 0.06054688 0.17013889]
 [0.47851562 0.27430556 0.05078125 0.13888889]
 [0.77734375 0.29166667 0.0703125  0.17013889]]
[[0.65039062 0.36310395 0.14257812 0.29282577]
 [0.19335938 0.19619327 0.13476562 0.27232796]]
[[0.18164062 0.2748538  0.09570312 0.43859649]
 [0.72851562 0.11111111 0.10546875 0.47368421]]
[

[[0.45410156 0.05273312 0.109375   0.13118971]]
[[0.140625   0.38020833 0.0703125  0.12239583]
 [0.37695312 0.44270833 0.06640625 0.125     ]
 [0.546875   0.45572917 0.0703125  0.12760417]
 [0.76367188 0.5        0.0625     0.13020833]]
[[0.41503906 0.12110482 0.26367188 0.25779037]]
[[0.41601562 0.09782105 0.15234375 0.09782105]]
[[0.34960938 0.07782101 0.22949219 0.23216602]]
[[0.37597656 0.1056683  0.20410156 0.20923723]]
[[0.16113281 0.234375   0.05273438 0.1015625 ]
 [0.26464844 0.30598958 0.04980469 0.0859375 ]
 [0.39941406 0.29557292 0.04785156 0.09505208]
 [0.51074219 0.29817708 0.05078125 0.09895833]
 [0.66503906 0.29817708 0.05664062 0.1015625 ]
 [0.80371094 0.31901042 0.06054688 0.11458333]]
[[0.36132812 0.13386881 0.19921875 0.42570281]]
[[0.32421875 0.10044643 0.0625     0.10491071]
 [0.75195312 0.08258929 0.05859375 0.12276786]]
[[0.43164062 0.05433901 0.25390625 0.32765612]]
[[0.34277344 0.05273438 0.26367188 0.24609375]]
[[0.32910156 0.13065693 0.34472656 0.34525547]]
[

[[0.29296875 0.32163743 0.203125   0.30409357]
 [0.00976562 0.38304094 0.15820312 0.30994152]]
[[0.03710938 0.49602122 0.10546875 0.18037135]
 [0.46875    0.36074271 0.06054688 0.11140584]]
[[0.53613281 0.15978695 0.05761719 0.09986684]
 [0.77734375 0.13315579 0.04980469 0.08788282]
 [0.87695312 0.23435419 0.03808594 0.05992011]
 [0.15722656 0.18375499 0.06347656 0.09720373]
 [0.35546875 0.33288948 0.03320312 0.04660453]]
[[0.13085938 0.4924406  0.15234375 0.43196544]]
[[0.42480469 0.44468391 0.29394531 0.28304598]
 [0.00292969 0.64942529 0.19140625 0.23204023]]
[[0.38085938 0.1972265  0.34667969 0.40215716]]
[[0.37988281 0.30238727 0.05664062 0.08090186]
 [0.47460938 0.40848806 0.05175781 0.08885942]
 [0.51953125 0.35013263 0.06445312 0.10477454]
 [0.71484375 0.36074271 0.06738281 0.11803714]]
[[0.1875     0.12890625 0.12890625 0.296875  ]
 [0.328125   0.171875   0.13867188 0.390625  ]
 [0.46875    0.19921875 0.12890625 0.4140625 ]
 [0.61328125 0.2890625  0.09765625 0.2890625 ]]
[[0.1

...

[[0.11523438 0.18502203 0.42382812 0.7958884 ]]
[[0.21972656 0.29073941 0.49902344 0.48169419]]
[[0.44140625 0.15812592 0.203125   0.3806735 ]]
[[0.45605469 0.07592472 0.12304688 0.10188189]]
[[0.18554688 0.04822335 0.02246094 0.02538071]
 [0.26757812 0.04568528 0.02636719 0.07360406]
 [0.25976562 0.15989848 0.0390625  0.06345178]
 [0.49316406 0.33248731 0.03125    0.12182741]]
[[0.22460938 0.18355185 0.55078125 0.67461263]]
[[0.07617188 0.27392739 0.13574219 0.0880088 ]]
[[0.3671875  0.0859375  0.31054688 0.44010417]]
[[0.37695312 0.25183016 0.16601562 0.31625183]]
[[0.4296875  0.26548673 0.1640625  0.33038348]]
[[0.25195312 0.21961933 0.33398438 0.4568082 ]]
[[0.50390625 0.20028612 0.13476562 0.24892704]]
[[0.29296875 0.25760456 0.33105469 0.45057034]]
[[0.40429688 0.190625   0.18554688 0.440625  ]]
[[0.5        0.04495114 0.15820312 0.16026059]]
[[0.29394531 0.13497729 0.21191406 0.18948735]]
[[0.48046875 0.36070381 0.109375   0.22580645]]
[[0.42480469 0.08966862 0.21386719 0.216374

[[0.39648438 0.16983895 0.14257812 0.33382138]]
[[0.40136719 0.07833089 0.140625   0.13836018]]
[[0.3046875  0.28119063 0.14160156 0.08613046]]
[[0.23828125 0.23133236 0.04785156 0.08784773]
 [0.36230469 0.17862372 0.05273438 0.09809663]]
[[0.29394531 0.10194805 0.20898438 0.30909091]
 [0.5234375  0.17012987 0.37695312 0.38116883]]
[[0.49023438 0.14769976 0.14257812 0.1283293 ]]
[[0.359375   0.22451994 0.23242188 0.22304284]]
[[0.26953125 0.11764706 0.0859375  0.20448179]
 [0.703125   0.13165266 0.09375    0.22128852]]
[[0.24609375 0.19097222 0.22460938 0.54861111]]
[[0.36425781 0.1132199  0.23632812 0.21465969]]
[[0.29492188 0.29282577 0.05664062 0.11420205]]
[[0.72460938 0.40932642 0.15429688 0.32124352]
 [0.29492188 0.45854922 0.17773438 0.29533679]]
[[0.36230469 0.12881098 0.30957031 0.35518293]]
[[0.47460938 0.06445312 0.12890625 0.12304688]]
[[0.24609375 0.15555556 0.0703125  0.17142857]
 [0.65429688 0.0952381  0.07617188 0.13650794]]
[[0.453125   0.04869298 0.15234375 0.10712455

[[0.23828125 0.05507745 0.3671875  0.88812392]]
[[0.29296875 0.00347222 0.48046875 0.94097222]]
[[0.27148438 0.16105417 0.05078125 0.10834553]]
[[0.45410156 0.18132366 0.19921875 0.23572076]]
[[0.3671875  0.1317716  0.22265625 0.46559297]]
[[0.24414062 0.26171875 0.41699219 0.4875    ]]
[[0.40625    0.15625    0.23242188 0.37109375]]
[[0.31152344 0.0742616  0.16894531 0.2185654 ]]
[[0.32226562 0.23133236 0.078125   0.17569546]]
[[0.25195312 0.30882353 0.07226562 0.15588235]
 [0.578125   0.31176471 0.06640625 0.17058824]]
[[0.34960938 0.0625     0.203125   0.53819444]]
[[0.15820312 0.25991189 0.16992188 0.24229075]]
[[0.2578125  0.16691069 0.1015625  0.22254758]]
[[0.5390625  0.15789474 0.13867188 0.23205742]]
[[0.3671875  0.17604712 0.2890625  0.28337696]]
[[0.40625    0.19791667 0.25390625 0.35677083]]
[[0.42871094 0.10949964 0.12109375 0.10732415]]
[[0.15234375 0.144      0.0703125  0.136     ]]
[[0.37207031 0.12518302 0.1171875  0.07247438]]
[[0.29980469 0.23060029 0.45605469 0.4838

[[0.23144531 0.16644909 0.40625    0.39099217]]
[[0.37109375 0.08318891 0.33984375 0.30155979]]
[[0.38867188 0.21081577 0.18261719 0.21081577]]
[[0.77636719 0.3810143  0.03125    0.04811443]
 [0.71875    0.41222367 0.03320312 0.05331599]
 [0.63769531 0.44213264 0.03320312 0.04941482]
 [0.59667969 0.39401821 0.03320312 0.04941482]
 [0.5390625  0.38751625 0.03125    0.04291287]
 [0.53222656 0.45903771 0.03125    0.04551365]
 [0.48730469 0.44473342 0.03125    0.04291287]
 [0.40722656 0.4473342  0.02832031 0.04291287]
 [0.33105469 0.4213264  0.02734375 0.04421326]
 [0.34472656 0.48244473 0.02929688 0.0390117 ]
 [0.27832031 0.44993498 0.02832031 0.04291287]
 [0.22363281 0.46423927 0.02929688 0.04161248]
 [0.15820312 0.47724317 0.02929688 0.04031209]]
[[0.28808594 0.12478032 0.30761719 0.37258348]]
[[0.38769531 0.14638158 0.16894531 0.20476974]]
[[0.375      0.20246914 0.22460938 0.41975309]]
[[0.3515625  0.56681351 0.12109375 0.14096916]
 [0.359375   0.4287812  0.09179688 0.10572687]]
[[0.4

[[0.453125   0.07783418 0.20019531 0.26734349]]
[[0.4296875  0.23357664 0.09179688 0.17226277]]
[[0.44726562 0.22580645 0.09765625 0.19941349]]
[[0.06640625 0.39647577 0.046875   0.09544787]
 [0.46679688 0.38325991 0.04882812 0.10866373]
 [0.25976562 0.36123348 0.04980469 0.11306902]
 [0.81835938 0.37885463 0.05957031 0.11453744]
 [0.6640625  0.40088106 0.05761719 0.11013216]]
[[0.32617188 0.23096447 0.29296875 0.50761421]]
[[0.30859375 0.15227736 0.26367188 0.23589395]]
[[0.32421875 0.25294118 0.28710938 0.54705882]]
[[0.328125   0.31551499 0.1484375  0.26336375]
 [0.6953125  0.36245111 0.12304688 0.19817471]]


In [28]:
from shutil import copyfile
from pathlib import Path

def create_dataset(data,dataset_type):
    if dataset_type == 'train':
        Images_folder = train_Images_folder
    else:
        Images_folder = test_Images_folder
    
    images_path = Path(data_folder+'/Images/'+dataset_type)
    images_path.mkdir(parents=True, exist_ok=True)
    labels_path = Path(data_folder+'/labels/'+dataset_type)
    labels_path.mkdir(parents=True, exist_ok=True)
    
    name = 1
    for path in data:
        copyfile(Images_folder+path, images_path/(str(name)+'.jpg'))
        with open(labels_path/(str(name)+'.txt'),'w') as out_file:
            for bbx in data[path]:
                x,y,w,h = [float(i) for i in bbx]
                out_file.write('1 '+str(x+w/2)+' '+str(y+h/2)+' '+str(w)+' '+str(h)+'\n')
        name+=1

In [29]:
create_dataset(train_data,'train')

In [30]:
create_dataset(test_data,'test')

In [31]:
def draw_bounding_boxes(im, bbxs,color='red',width=3):
    draw = ImageDraw.Draw(im)
    l,w,_ = [float(i) for i in np.array(im).shape]
    for box in bbxs:
        x0 = (box[1]-box[3]/2)*w
        y0 = (box[2]-box[4]/2)*l
        x1 = (box[1]+box[3]/2)*w
        y1 = (box[2]+box[4]/2)*l
        draw.rectangle([x0,y0,x1,y1], fill=None, outline=color, width=width)
    return(im)

image_name = '1600'
bbxs = []
with open(data_folder+'/labels/train/'+image_name+'.txt', 'r') as file:
    while True:
        line = file.readline()
        if not line: break
        line = np.fromstring(line, dtype=float, sep=' ')
        
        bbxs.append(line)

im = Image.open(data_folder+'/Images/train/'+image_name+'.jpg')
im = draw_bounding_boxes(im,bbxs)
im.show()